In [1]:
import tensorflow.keras,sklearn
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import tensorflow.compat.v1 as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
seed=4
np.random.seed(seed)
tf.set_random_seed(seed)
import matplotlib.pyplot as plt
import cv2
import pickle
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import requests
import io
from PIL import Image, ImageTk
import tkinter as tk
from tkinter.filedialog import asksaveasfilename
from tkinter.filedialog import askopenfilename
from numpy import asarray
import time
import pathlib
print(pathlib.Path().absolute())
#should run on 2.4.0
print(tf.__version__)

C:\Users\Warsh\ML_Notebooks_edited\py895_quickDraw-main (3)\py895_quickDraw-main
2.4.0


# Unrestricted Comparison
This is the notebook in which we compared the models with unrestricted training time and parameters.

In [2]:
all_cat_titles = ['airplane','monalisa','dragon','giraffe','axe','banana','eiffeltower','snail','windmill','snowman','schoolbus','smileyface','lollipop',]

img_rows, img_cols = 28, 28 

use_dataset_titles = ['X_use','Y_use']
ot = ['X_onlytest','Y_onlytest']
#DONT YOU DARE TOUCH THE ONLYTEST FILES.
#IF YOU ARE TRAINING A NEURAL NETWORK, TRAIN, VALIDATE, AND TEST FROM THE 'USE' FILES.

#load data
url_main = 'https://physics.bu.edu/~warsh/ML_QD/'
datalib={}

#loading samples
response = requests.get(url_main + 'ot1' +'.npy')
response.raise_for_status()
data = np.load(io.BytesIO(response.content), allow_pickle=False)
data = data.reshape(data.shape[0], img_rows*img_cols)
datalib[ot[0]] = data

response = requests.get(url_main + 'ot2' +'.npy')
response.raise_for_status()
data = np.load(io.BytesIO(response.content), allow_pickle=False)
datalib[ot[1]] = data

num_classes = int(max(datalib['Y_onlytest'])+1)
cat_titles = all_cat_titles[:num_classes]
print('loaded:',cat_titles)

loaded: ['airplane', 'monalisa', 'dragon', 'giraffe', 'axe', 'banana', 'eiffeltower', 'snail', 'windmill', 'snowman']


In [4]:
#shape test set
Y_ot = datalib['Y_onlytest']
X_ot = datalib['X_onlytest']

#reshape into 2D
X_ot = X_ot.reshape(X_ot.shape[0],img_rows,img_cols)

#yeah okay some floats
X_ot = X_ot.astype('float32')

#scale onto the fun zer-O to O-ne inverval
X_ot /= 255
X_ot_IS = X_ot
# converting the actual numbers to the the one-hot vectors
Y_ot = tensorflow.keras.utils.to_categorical(Y_ot, num_classes)

#reshape data, depending on Keras backend, final data formatting
if tensorflow.keras.backend.image_data_format() == 'channels_first':
    X_ot = X_ot.reshape(X_ot.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_ot = X_ot.reshape(X_ot.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

#more debugging shit
print('X_ot shape:', X_ot.shape)
print('Y_ot shape:', Y_ot.shape)

X_ot shape: (337168, 28, 28, 1)
Y_ot shape: (337168, 10)


In [5]:
#load model from folder
possible_models = ['warshenstein','anna_banana','chris_is_a_model','jiinnet']
workdir = str(pathlib.Path().absolute())+'\\'
models={}
for mod in possible_models:
    models[mod]=load_model(workdir+mod+'.h5')
#score dictionary, I recently started using dictionaries, they're so fun.
scoreC={}

## Results with these loaded models:

In [6]:
for key in models.keys():
    scoreC[key]  = models[key].evaluate(X_ot, Y_ot, verbose=1)
for key in scoreC.keys():
    print(key + ' Accuracy: %.4f'%scoreC[key][1])
    print(key + ' Loss: %.4f'%scoreC[key][0])

10537/10537 [==============================] - 65s 6ms/step - loss: 0.1489 - accuracy: 0.9673
warshenstein Accuracy: 0.9709
warshenstein Loss: 0.1073
anna_banana Accuracy: 0.9598
anna_banana Loss: 0.1610
chris_is_a_model Accuracy: 0.9621
chris_is_a_model Loss: 0.1420
jiinnet Accuracy: 0.9673
jiinnet Loss: 0.1489
